## Query

In [27]:
from llama_index.core import StorageContext,load_index_from_storage
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import nest_asyncio
nest_asyncio.apply()

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large",device='cpu')
custom_api = 'float16-ZrIyRfu4XLpUS5y4ZUOd80VLpSjdYUfsLBY3kSUDTMZB8ra0p1'
custom_url = 'https://api.float16.cloud/v1/llamaindex/'
llm = OpenAILike(api_base=custom_url,api_key=custom_api,max_tokens=100,model='SeaLLM-7b-v2',api_version='v1')

load_index = load_index_from_storage(StorageContext.from_defaults(persist_dir='./localstore/'),embed_model=embed_model)

In [28]:
retrieval_engine = load_index.as_retriever(similarity_top_k = 5)
response = retrieval_engine.retrieve("FinOps คืออะไร?")
print('-------------RAG-------------')
print(response)

-------------RAG-------------
[NodeWithScore(node=TextNode(id_='f6f1d2f7-b70f-4ce8-afd4-82594f15764e', embedding=None, metadata={'filename': 'finops.md', 'extension': '.md'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='29c14040-3d75-4e8c-b397-7181757629b8', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'filename': 'finops.md', 'extension': '.md'}, hash='d5fbef9aa2cb4c27540f17fbc495a4e8c0c3b06af253f70394eeeb3e34a07ba9'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='0bbd28f9-ba8e-4887-bed1-8473640ade3c', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='c18455576ae9bbffa7d79e1009f948ab3675f4f62afec647200656774c91f0b0')}, text='II FinOps ?  \nA FinOps (cloud Financial Operations)\nFinOps Foundation เป็น Foundation ที่ได้รับการสนับสนุนจาก The Linux Foundation ในการช่วยสร้าง Foundation สำหรับการส่งเสริมและผลักดันการใช้งาน Cloud computing อย่างมีคุณค่า\n\nB History\nFinOps Foundation ก่

In [29]:
query_engine = load_index.as_query_engine(similarity_top_k = 5,llm=llm)
response = query_engine.query("FinOps คืออะไร?")
print('-------------RAG-------------')
print(response)

-------------RAG-------------
FinOps, or Cloud Financial Operations, is a strategic approach to managing cloud costs and resources. It involves analyzing, managing, and optimizing cloud usage to achieve cost efficiency and resource utilization. This includes forecasting costs, setting budgets, and monitoring usage to ensure that resources are being used effectively.

The maturity of a FinOps practice can be divided into three stages: Crawl, Walk, and Run. 

1. Crawl: This is the initial stage


In [30]:
query_engine = load_index.as_chat_engine(similarity_top_k = 5,llm=llm)
response = query_engine.chat("FinOps คืออะไร?")
print('-------------RAG-------------')
print(response)

-------------RAG-------------
FinOps, or Cloud Financial Operations, is a strategic approach to managing cloud costs and resources. It involves analyzing, managing, and optimizing cloud usage to ensure that organizations can use resources efficiently and cost-effectively. This includes understanding the costs of cloud services, monitoring usage, and making data-driven decisions to optimize spending. FinOps is not a one-size-fits-all approach. It


In [31]:
response = query_engine.chat("ตอบภาษาไทยได้มั้ย?")
print('-------------RAG-------------')
print(response)

-------------RAG-------------
FinOps หรือ Cloud Financial Operations คือ แนวทางการจัดการค่าใช้จ่ายและทรัพยากรของคลาวด์คอมพิวติง โดยมีวัตถุประสงค์เพื่อให้องค์กรสามารถใช้ทรัพยากรได้อย่างมีประสิทธิภาพและประหยัดค่าใช้จ่าย การ FinOps ประกอบด้วยการวิเคราะห์ค่าใช้จ่ายของบริการค


#### RAG

In [32]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer,load_index_from_storage,StorageContext
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response_synthesizers.type import ResponseMode
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai_like import OpenAILike

# configure LLMs
custom_api = 'float16-ZrIyRfu4XLpUS5y4ZUOd80VLpSjdYUfsLBY3kSUDTMZB8ra0p1'
custom_url = 'https://api.float16.cloud/v1/llamaindex/'
llm = OpenAILike(api_base=custom_url,api_key=custom_api,max_tokens=512,model='SeaLLM-7b-v2',api_version='v1')
embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large",device='cpu')

# build index
load_index = load_index_from_storage(StorageContext.from_defaults(persist_dir='./localstore/'),embed_model=embed_model)


# load_index.as_retriever()

# configure retriever
retriever = VectorIndexRetriever(
    index=load_index, # type: ignore
    similarity_top_k=2,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode=ResponseMode.SIMPLE_SUMMARIZE,
    llm = llm
)


# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query("FinOps ประกอบด้วยอะไรบ้าง?")
print(response)

FinOps, or Financial Operations, is a practice that focuses on managing cloud costs and optimizing the use of cloud resources. It involves a combination of people, processes, and technology to ensure that cloud spending aligns with business objectives. Here's a breakdown of what FinOps typically includes:

1. **People (Personas):** FinOps involves various personas, each with their own roles and responsibilities. These include:
   - **FinOps Practitioner:** Responsible for implementing FinOps practices within the organization.
   - **Executive:** Responsible for providing strategic direction and ensuring alignment with business goals.
   - **Business:** Responsible for understanding customer needs and ensuring the cloud solutions meet those needs.
   - **Finance:** Responsible for managing the financial aspects of cloud spending.
   - **Engineer:** Responsible for ensuring the technical feasibility of cloud solutions.

2. **Processes:** FinOps involves a set of processes that help manag

In [33]:
from llama_index.core import VectorStoreIndex,get_response_synthesizer
from llama_index.readers.file import FlatReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai_like import OpenAILike
from llama_index.core.node_parser import SimpleFileNodeParser,TokenTextSplitter
from llama_index.core import Settings
from transformers import AutoTokenizer
from pathlib import Path

custom_api = 'float16-ZrIyRfu4XLpUS5y4ZUOd80VLpSjdYUfsLBY3kSUDTMZB8ra0p1'
custom_url = 'https://api.float16.cloud/v1/llamaindex/'
tokenizer = AutoTokenizer.from_pretrained("SeaLLMs/SeaLLM-7B-v2").encode
llm = OpenAILike(api_base=custom_url,
                 api_key=custom_api,
                 max_tokens=512,
                 model='SeaLLM-7b-v2',
                 api_version='v1',
                 )
token_spliter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=64,
    tokenizer=tokenizer,
)
text_parser = SimpleFileNodeParser()

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large",device='cpu')
md_docs = FlatReader().load_data(Path("./MockupData/finops.md"))
md_nodes = token_spliter.get_nodes_from_documents(text_parser.get_nodes_from_documents(md_docs)) # type: ignore

Settings.embed_model = embed_model
Settings.llm = llm
Settings.context_window = 4000 
Settings.tokenizer = tokenizer
Settings.num_output = 512
response_synthesizer = get_response_synthesizer(
      response_mode="simple_summarize" # type: ignore
)

index = VectorStoreIndex(md_nodes)
query_engine = index.as_query_engine(
    llm=llm,
    similarity_top_k = 10, #8000 token ; Expect 10 => 5 ; Actual 10 (100%) => 10 (50%)
    response_synthesizer = response_synthesizer)
response = query_engine.query("FinOps คืออะไร?")

In [34]:
response.response # type: ignore

'FinOps (Cloud Financial Operations) คือการบริหารจัดการค่าใช้จ่ายและการใช้งานของคลาวด์คอมพิวติงในองค์กร โดยมีเป้าหมายเพื่อลดค่าใช้จ่ายที่ไม่จำเป็นและเพิ่มประสิทธิภาพการใช้งานคลาวด์ คำว่า FinOps มาจากคำว่า Financial Operations ซึ่งเป็นการบริหารจัดการด้านการเงินและการเงินในองค์กร โดย FinOps จะเน้นไปที่การบริหารจัดการค่าใช้จ่ายและการใช้งานคลาวด์คอมพิวติงในองค์กร\n\nFinOps Foundation เป็นองค์กรที่สนับสนุนการพัฒนาและส่งเสริมการใช้งาน FinOps ในองค์กร โดยมีเป้าหมายเพื่อช่วยให้องค์กรต่างๆ สามารถใช้งานคลาวด์คอมพิวติงได้อย่างมีประสิทธิภาพและมีค่าใช้จ่ายที่เหมาะสม\n\nการใช้งาน FinOps จะช่วยให้องค์กรสามารถติดตามและควบคุมการใช้งานคลาวด์ได้อย่างมีประสิทธิภาพ โดยการใช้เครื่องมือและแนวทางการทำงานที่เหมาะสม เพื่อให้สามารถตรวจสอบและจัดการค่าใช้จ่ายของคลาวด์ได้อย่างแม่นยำและมีประสิทธิภาพ\n\nการใช้งาน FinOps จะช่วยให้องค์กรสามารถลดค่าใช้จ่ายที่ไม่จำเป็นและเพิ่มประสิทธิภาพการใช้งานคลาวด์ได้ โดยการใช้เครื่องมือและแนวทางการทำงานที่เหมาะสม เพื่อให้สามารถตรวจสอบและจัดการค่าใช้จ่ายของคลาวด์ได้อย่างแม่นยำและมีปร

## Agent

In [35]:
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai_like import OpenAILike
from llama_index.core.agent import ReActAgent
from llama_index.core.agent.react.formatter import ReActChatFormatter

custom_api = 'float16-ZrIyRfu4XLpUS5y4ZUOd80VLpSjdYUfsLBY3kSUDTMZB8ra0p1'
custom_url = 'https://api.float16.cloud/v1/llamaindex/'

# define sample Tool
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

# initialize llm
llm = OpenAILike(api_base=custom_url,api_key=custom_api,max_tokens=100,model='SeaLLM-7b-v2',api_version='v1')

# initialize ReAct agent
agent = ReActAgent.from_tools([multiply_tool], llm=llm, verbose=True)
res = agent.chat("What is 2123 * 111")
print(res)

Thought: I need to use a tool to help me answer the question.
Action: multiply
Action Input: {"a": 2123, "b": 111}

Observation: 237033

Thought: I can answer without using any more tools.
Answer: 237033
237033


In [36]:
prompt = ReActChatFormatter().format(tools=[multiply_tool],chat_history=[])
for i in prompt : 
    print(i)

system: 
You are designed to help with a variety of tasks, from answering questions     to providing summaries to other types of analyses.

## Tools
You have access to a wide variety of tools. You are responsible for using
the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools
to complete each subtask.

You have access to the following tools:
> Tool Name: multiply
Tool Description: multiply(a: int, b: int) -> int
Multiple two integers and returns the result integer
Tool Args: {"type": "object", "properties": {"a": {"title": "A", "type": "integer"}, "b": {"title": "B", "type": "integer"}}, "required": ["a", "b"]}


## Output Format
To answer the question, please use the following format.

```
Thought: I need to use a tool to help me answer the question.
Action: tool name (one of multiply) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {"i

### Tools

#### Query Engine Tools

In [37]:
from llama_index.core import VectorStoreIndex,StorageContext
from llama_index.readers.file import FlatReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.storage.index_store import SimpleIndexStore
from pathlib import Path

sentence_spilit = SentenceSplitter(
    chunk_size=512
)

storage_context = StorageContext.from_defaults(
    vector_store=SimpleVectorStore(),
    docstore=SimpleDocumentStore(),
    index_store=SimpleIndexStore(),
)


embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large",device='cpu')
finops_docs = FlatReader().load_data(Path("./MockupData/finops.md"))
orbit_docs = FlatReader().load_data(Path("./MockupData/orbit.md"))

finops_nodes = sentence_spilit.get_nodes_from_documents(finops_docs)
orbit_nodes = sentence_spilit.get_nodes_from_documents(orbit_docs)

finops = VectorStoreIndex(finops_nodes,embed_model=embed_model,storage_context=storage_context)
orbit = VectorStoreIndex(orbit_nodes,embed_model=embed_model,storage_context=storage_context)

In [38]:
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.tools import QueryEngineTool,ToolMetadata
from llama_index.llms.openai_like import OpenAILike

custom_api = 'float16-ZrIyRfu4XLpUS5y4ZUOd80VLpSjdYUfsLBY3kSUDTMZB8ra0p1'
custom_url = 'https://api.float16.cloud/v1/llamaindex/'

# initialize llm
llm = OpenAILike(api_base=custom_url,api_key=custom_api,max_tokens=1024,model='SeaLLM-7b-v2',api_version='v1')

finops_engine = finops.as_query_engine(similarity_top_k=3,llm=llm)
orbit_engine = orbit.as_query_engine(similarity_top_k=3, llm=llm)

query_engine_tools = [
    QueryEngineTool(
        query_engine=finops_engine,
        metadata=ToolMetadata(
            name="finops",
            description="ข้อมูลเกี่ยวกับแนวคิดของ Finops",
        ),
    ),
    QueryEngineTool(
        query_engine=orbit_engine,
        metadata=ToolMetadata(
            name="orbit",
            description="ข้อมูลเกี่ยวกับแนวคิดของ Orbit",
        ),
    ),
]

sub_query_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools, llm=llm, verbose=True,use_async=True)

In [39]:
import nest_asyncio
nest_asyncio.apply()
res = sub_query_engine.query('finops และ orbit แตกต่างกันอย่างไร')

Generated 2 sub questions.
[finops] Q: What is Finops
[orbit] Q: What is Orbit
[finops] A: FinOps, or cloud financial operations, is a framework and set of best practices for managing and optimizing cloud costs. It's a collaborative effort between finance, IT, and lines of business to manage cloud resources and costs. The goal of FinOps is to ensure that cloud resources are used efficiently and cost-effectively, while also providing visibility and control over cloud spending.

FinOps involves several key elements, including cost allocation, cost optimization, and cost governance. It also involves understanding the cost drivers of cloud usage, such as usage patterns, resource utilization, and pricing models.

The FinOps Foundation, which is supported by The Linux Foundation, aims to promote the adoption and advancement of FinOps practices. It provides a community for practitioners to share knowledge, best practices, and tools.

In summary, FinOps is a strategic approach to managing clou

In [40]:
res.response 

"FinOps and Orbit are two distinct concepts that serve different purposes.\n\nFinOps, or cloud financial operations, is a framework and set of best practices for managing and optimizing cloud costs. It's a collaborative effort between finance, IT, and lines of business to manage cloud resources and costs. The goal of FinOps is to ensure that cloud resources are used efficiently and cost-effectively, while also providing visibility and control over cloud spending.\n\nOn the other hand, Orbit Model is a framework or model used to measure and understand the level of engagement and impact of individuals within a community or organization. It is designed to help community managers and creators enhance diversity and improve the effectiveness of their communities.\n\nIn summary, FinOps is a strategic approach to managing cloud costs, ensuring that organizations are getting the most value from their cloud investments, while Orbit Model is a framework to measure and understand the level of enga

In [41]:
res = sub_query_engine.query('finops คืออะไร')

Generated 2 sub questions.
[finops] Q: Finops คืออะไร
[orbit] Q: Orbit คืออะไร
[orbit] A: Orbit Model เป็นกรอบการทำงานที่ใช้ในการสร้างชุมชนที่มีอัตราการมีส่วนร่วมสูง (high gravity communities) โดยเน้นการสร้างประสบการณ์สมาชิกที่โดดเด่นและเสริมสร้างความสัมพันธ์ระหว่างสมาชิกในชุมชน

โมเดลนี้แบ่งออกเป็นสี่แนวคิดหลัก ได้แก่ "Gravity" (อัตราการมีส่วนร่วมของสมาชิกที่วัดที่ระดับของชุมชน), "Love" (การมีส่วนร่วมของสมาชิกรายบุคคล), "Reach" (ขนาดของผลกระทบที่สมาชิกมีต่อความสำเร็จของชุมชน), และ "Impact" (ผลลัพธ์ที่เกิดขึ้นจากการดำเนินการ).

Orbit Model เน้นการวัดและเข้าใจกิจกรรมต่างๆ ที่สมาชิกมีส่วนร่วมในชุมชน โดยแบ่งประเภทกิจกรรมต่าง ๆ ออกเป็นกลุ่มต่าง ๆ โดยมีน้ำหนักที่บ่งบอกระดับความรับผิดชอบและการมีส่วนร่วมในชุมชน ตัวอย่างของกิจกรรมเช่น การเข้าร่วม, การโพสต์ในฟอรัม, การนำเสนอ, การจัดการพบปะ, การถาม-ตอบคำถาม, การเปิด pull request ในแพลตฟอร์มเช่น GitHub, การเขียนบทความบล็อก, การบริจาค, และการส่งคำขอบคุณ.

โมเดลนี้เป็นเครื่องมือที่มีประสิทธิภาพสำหรับผู้จัดการและผู้สร้างชุมชน เพื่อเสริมความหลากหลายแ

In [42]:
res.response

'FinOps (Cloud Financial Operations) เป็นโครงการที่ได้รับการสนับสนุนจาก The Linux Foundation เพื่อช่วยสร้างโครงสร้างพื้นฐานที่สนับสนุนการใช้งาน Cloud computing อย่างมีคุณค่า. มันถูกสร้างขึ้นหลังจากการมีอยู่ของ Cloud computing ประมาณ 10 ปี (AWS establish in 2006) และถือว่าเป็นโครงสร้างพื้นฐานที่อายุน้อยมาก (เทียบกับปี 2023).\n\nFinOps มีวัตถุประสงค์เพื่อช่วยให้บริษัทสามารถจัดการกับการใช้จ่ายใน Cloud computing ได้อย่างมีประสิทธิภาพ โดยการสร้างกระบวนการทำงานและรายงานผลที่มีประสิทธิภาพ. มันช่วยให้ทีมต่างๆ เช่น ทีม Engineer, ทีม Finance, และทีม Procurement สามารถทำงานร่วมกันได้อย่างมีประสิทธิภาพ.\n\nFinOps ยังมีแนวคิดเกี่ยวกับ "Maturity" ซึ่งแบ่งออกเป็น 3 ช่วง ได้แก่ Crawl, Walk, และ Run. ช่วง Crawl คือช่วงเริ่มต้นของการนำ FinOps เข้ามาประยุกต์เข้ากับบริษัทของตัวเอง. ช่วง Walk คือช่วงของการทำงานที่มีความซับซ้อนมากขึ้นและมีการใช้งานระบบอัตโนมัติ. ช่วง Run คือช่วงที่การทำงานเป็นไปอย่างราบรื่นและมีประสิทธิภาพ.\n\nนอกจากนี้ FinOps ยังมี "Personas" หรือบุคคลที่เกี่ยวข้องกับการใช้งาน FinOps ซึ่งร